# Green PPA Pricing - Expected Value

In [6]:
import sys
import datetime as dt
sys.path.append('../../../..')
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger('rivapy.pricing')
logger.setLevel(logging.DEBUG)

from rivapy.instruments import SimpleSchedule,  GreenPPASpecification
from rivapy.marketdata import PowerPriceForwardCurve
import rivapy.sample_data.residual_demand_models as rdm_sample
from rivapy.pricing.pricing_request import GreenPPAPricingRequest
from rivapy.pricing import green_ppa_mean_pricer

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Green PPA Specification

In [37]:
simple_schedule = SimpleSchedule(dt.datetime(2022,12,1), dt.datetime(2023,12,1,4,0,0), freq='1H')
green_ppa = GreenPPASpecification(simple_schedule, 
                                  fixed_price = 10.0, 
                                  max_capacity=10, 
                                  technology = 'wind', 
                                  udl = 'power',
                                  location='')

In [41]:
pr = GreenPPAPricingRequest(theo_val=True, cf_expected=True, cf_paths=True)
rd_model = rdm_sample.WagnerModel.residual_demand_model(capacity_wind=25, capacity_solar=20)

In [42]:
results = green_ppa_mean_pricer.price(dt.datetime(2022,12,1), green_ppa, rd_model, pr, n_sims=10, seed=42)

/home/doeltz/doeltz/development/RiVaPy/rivapy/tools/datetime_grid.py:24: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  self.dates = pd.date_range(start, end, freq=freq, tz=tz, closed=closed).to_pydatetime()


In [43]:
results

{'theo_val': 106.33955937000052,
 'cf_expected': array([88.65277778, 73.38856324, 81.46445778, ..., 35.01461317,
        16.81824448, 35.65894031]),
 'cf_paths': array([[ 88.65277778,  88.65277778,  88.65277778, ...,  88.65277778,
          88.65277778,  88.65277778],
        [120.59769197,  73.59260888,  85.56582072, ...,  56.00689862,
         124.17623418,  70.07224311],
        [150.66679196,  55.10256533,  27.52760532, ..., -50.1915637 ,
         153.27188838, 117.63575482],
        ...,
        [ 41.34946064,  80.04721905,  89.40655705, ..., 144.98860263,
          52.47875851,  31.48219098],
        [ 41.83472586,  82.56825408,  91.22650758, ..., 150.22908045,
          42.48726577,  27.08113649],
        [ 44.12449773,  52.31010233, 117.90576107, ..., 101.5151421 ,
          36.99336384,  30.78853215]])}

In [ ]:
values = np.ones((len(simple_schedule.get_schedule()),))
hpfc = PowerPriceForwardCurve( dt.datetime(2022,12,1), dt.datetime(2022,12,1), 
                              dt.datetime(2023,12,1,4,0,0), freq='1H', values = values)